## Libraries

In [ ]:
import os, sys, glob, re
from osgeo import gdal, osr, ogr
from datetime import datetime, timedelta, date, timezone
import numpy as np
from joblib import Parallel, delayed
import io
import operator
import rasterio as rio
import pandas as pd
import geopandas as gpd
import pyproj
import ast
from shapely.geometry import Polygon
from pyproj import Transformer



In [ ]:
import Global_Functions
import Gdal_Functions

import ee
ee.Authenticate(force=False)

## 1) NBR, NBR2

In [ ]:
BA = ee.ImageCollection('projects/ee-fireccihrba/assets/SFD_BA/Siberia/PeatFire/SIN_1km')
studyArea = ee.FeatureCollection('users/fireccihrba/BAMT/Regions/Siberia').geometry()
grid2d = ee.FeatureCollection("users/fireccihrba/BAMT/Regions/Tiles_Siberia_3dLon")
grid4d = ee.FeatureCollection("users/fireccihrba/BAMT/Regions/Tiles_Siberia_4dLat")
BA30 = ee.ImageCollection('projects/ee-fireccihrba/assets/SFD_BA/Siberia/PeatFire/JD')
print(BA.filter(ee.Filter.eq('system:index', 'BA_Siberia_Lndst_2002_JD_1km')).getInfo())
print(f"\n\n{BA30.first().getInfo()}")
print(f"\n\n{grid4d.first().getInfo()}")

In [ ]:
def download_image(image, name, export_folder, geo, proj, scale, transform=None, to='Drive',  logfile=False):
    if transform:
        scale = None
    if to == 'Drive':
        task = ee.batch.Export.image.toDrive(image=image, 
                              description=name, 
                              folder=export_folder, 
                              crs=proj,  
                              scale=scale,
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)  
    else:
        task = ee.batch.Export.image.toAsset(image=image, 
                              description=name, 
                              assetId=f'{export_folder}/{name}', 
                              crs=proj,  
                              scale=scale, 
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)              
    task.start()
    Global_Functions.print_and_log(logfile, f"\t\t +++ Exporting {name} ...")
    
def calculate_NBR(year, tile, sensor='Landsat', logfile=False):
    if not logfile:
        logfile = io.StringIO()
    # Global_Functions.print_and_log(logfile, f'\n\t\t{15 * "-"} {tile} {15 * "-"}')        
    studyArea = tile.geometry()
    tilename = tile.get('TILE').getInfo()

    """----- Main functions ------"""
    def get_indices(image):
        nbr = image.normalizedDifference(['SWIR2', 'NIR']).multiply(10000).int16()
        nbr2 = image.normalizedDifference(['SWIR2', 'SWIR1']).multiply(10000).int16()
        return image.int16().addBands([nbr.rename(['NBR']), nbr2.rename(['NBR2'])])

    def mask_landsat(image):
        date = ee.Number.parse(ee.Date(image.get('system:time_start')).format('yyyyDDD'))
        mask = image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(3).int()).eq(0) \
            .And(image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(4).int()).eq(0)) \
            .And(image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(2).int()).eq(0)) \
            .And(image.select('QA_PIXEL').bitwiseAnd(ee.Number(2).pow(5).int()).eq(0))
        satellite = ee.String(image.get('SPACECRAFT_ID'))
        image = ee.Image(ee.Algorithms.If(
        satellite.compareTo('LANDSAT_4').eq(0) \
          .Or(satellite.compareTo('LANDSAT_5').eq(0)) \
          .Or(satellite.compareTo('LANDSAT_7').eq(0)),
        image.select(['SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) \
          .multiply(0.0000275).add(-0.2).multiply(10000) \
          .rename(['Red', 'NIR', 'SWIR1', 'SWIR2']),
        image.select(['SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']) \
          .multiply(0.0000275).add(-0.2).multiply(10000) \
          .rename(['Red', 'NIR', 'SWIR1', 'SWIR2'])
        ))
        image = image.updateMask(mask.eq(1))
        image = image.updateMask(image.select('NIR').gt(50).And(image.select('Red').gt(50)))
            ## additional mask for clouds missed by CFMask, here NIR is more restrictive
        image = image.updateMask(image.select('SWIR2').lt(3000).And(image.select('NIR').lt(2600)))             
        return image.clip(studyArea)
    
    def mask_MOD09(image):
        mask = image.select('state_1km').bitwiseAnd(ee.Number(2).pow(10).int()).eq(0)  \
              .And(image.select('state_1km').bitwiseAnd(ee.Number(2).pow(15).int()).eq(0)) ## Snow   
        image = image.select(['sur_refl_b02', 'sur_refl_b06', 'sur_refl_b07']) \
                    .rename(['NIR', 'SWIR1', 'SWIR2'])        
        return image.updateMask(mask)
        
    """----- Processing ------"""

    if sensor == 'Landsat':
        date_1 = f'{int(year)-2}-03-01'
        date_2 = f'{int(year)-1}-12-01'
        date_2_2 = f'{int(year)}-03-01'
        date_3 = f'{int(year)+1}-12-01'
        pre_image = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').filterBounds(studyArea).filterDate(date_1, date_2) \
            .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(studyArea).filterDate(date_1, date_2)) \
            .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(studyArea).filterDate(date_1, date_2)) \
            .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(studyArea).filterDate(date_1, date_2)) \
            .map(mask_landsat).map(get_indices).select(['NBR', 'NBR2'])

        post_image = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').filterBounds(studyArea).filterDate(date_2_2, date_3) \
            .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(studyArea).filterDate(date_2_2, date_3)) \
            .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(studyArea).filterDate(date_2_2, date_3)) \
            .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(studyArea).filterDate(date_2_2, date_3)) \
            .map(mask_landsat).map(get_indices).select(['NBR', 'NBR2'])
    else:
        date_1 = f'{int(year)-1}-03-01'
        date_2 = f'{int(year)-1}-12-01'
        date_2_2 = f'{int(year)}-03-01'
        date_3 = f'{int(year)}-12-01'
        pre_image = ee.ImageCollection("MODIS/061/MOD09GA").filterDate(date_1, date_2).map(
            mask_MOD09).map(get_indices).select(['NBR', 'NBR2'])
        post_image = ee.ImageCollection("MODIS/061/MOD09GA").filterDate(date_2_2, date_3).map(
            mask_MOD09).map(get_indices).select(['NBR', 'NBR2'])
    
    proj = 'EPSG:4326'
    pre_image = pre_image.median().rename(['NBR', 'NBR2'])
    post_image = post_image.qualityMosaic('NBR')
    bandtype = post_image.bandTypes().get('NBR')
    diff_image = post_image.rename(['post_NBR', 'post_NBR2']).addBands(
                   post_image.subtract(pre_image).rename(['diff_NBR', 'diff_NBR2'])) \
                   .cast({'diff_NBR': bandtype, 'diff_NBR2': bandtype})

    # BA_image = BA.filter(ee.Filter.eq('system:index', f'BA_Siberia_Lndst_{year}_JD_1km')).first()
    BA_image = BA30.filter(ee.Filter.eq('system:index', 
                f'BAMT_BA_Siberia_Lndst_{year}_TILE-{tilename}_JD_Correct_Patches')).first()
    BA_image = BA30.filter(ee.Filter.stringContains('system:index', 
                f'{year}')).mosaic()
    # print(BA_image.getInfo())
    diff_image = diff_image.updateMask(BA_image.gt(0))
    # print(diff_image.getInfo())
    gt = ee.String(tile.get('transform'))
    gt = gt.slice(1, -1).split(',').map(lambda x: ee.Number.parse(x))
    # print(gt.getInfo())
    download_image(diff_image, f'NBR_{year}_{tilename}', 'PeatFire', studyArea, proj, scale=None, 
                   transform=gt, logfile=logfile)
  

In [ ]:
calculate_NBR(2023, studyArea)

In [ ]:
Tiles = grid4d.aggregate_array('TILE').getInfo()
for year in list(range(2023, 2000, -1))[:]:
    print(year)
    for tile in Tiles[:]:
            tile = grid4d.filter(ee.Filter.eq('TILE', tile)).first()
            # print(tile.getInfo())
            calculate_NBR(year, tile)

## 2) SoilGrids

In [ ]:
local_path = '/media/amin/STORAGE/STORAGE/OneDrive/PhD/Landsat_BA'
sib3d = gpd.read_file(f'{local_path}/Regions/Tiles_Siberia_3dLon.shp')
layers = ['sand', 'silt', 'clay']
""" The coefficients should ideally be [0-5]: 1/6, [5-15]: 1/3, [15-30]: 1/2 
    However, since the fire attacks the top layers first we use 1/3 for all"""
res = 250
igh = "+proj=igh +lat_0=0 +lon_0=0 +datum=WGS84 +units=m +no_defs" # proj string for Homolosine projection
transformer = Transformer.from_crs("epsg:4326", pyproj.CRS.from_proj4(igh), always_xy=True)


for i, row in enumerate(list(sib3d.iterrows())[:]):
    tile = 'TILE-' + row[1]['TILE']
    poly = row[1]['geometry']
    poly_igh = Polygon([transformer.transform(x, y) for x, y in poly.exterior.coords]) 
    gt = ast.literal_eval(row[1]['transform'])
    ## xmin, ymax, xmax, ymin
    bb = poly_igh.bounds
    bb = bb[0], bb[3], bb[2], bb[1]
    print(i+1, tile, bb)
    # bb = (-337500.000,1242500.000,152500.000,527500.000)
    location = "https://files.isric.org/soilgrids/latest/data/"
    sg_url = f"/vsicurl?max_retry=3&retry_delay=1&list_dir=no&url={location}"
    
    kwargs = {'format': 'GTiff', 'projWin': bb, 'projWinSRS': igh, 'xRes': res, 'yRes': res, 
              'creationOptions': ["TILED=YES", "COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES"]}
    for l in layers[:]:
        print(l)
        ds0_5 = gdal.Translate(f'/vsimem/{l}_0-5cm_igh.tif', 
                        sg_url + f'{l}/{l}_0-5cm_mean.vrt', 
                        **kwargs).ReadAsArray()
        ds5_15 = gdal.Translate(f'/vsimem/{l}_5-15cm_igh.tif', 
                        sg_url + f'{l}/{l}_5-15cm_mean.vrt', 
                        **kwargs).ReadAsArray()
        ds15_30 = gdal.Translate(f'/vsimem/{l}_15-30cm_igh.tif', 
                        sg_url + f'{l}/{l}_15-30cm_mean.vrt', 
                        **kwargs).ReadAsArray()
        with rio.open(f'/vsimem/{l}_0-5cm_igh.tif') as src:
            profile = src.profile
        [gdal.Unlink(f'/vsimem/{l}_{i}-{j}cm_igh.tif') for (i, j) in zip([0, 5, 15], [5, 15, 30])]
        arr = np.nanmean(np.stack([ds0_5, ds5_15, ds15_30], axis=2), axis=2)
        del ds15_30, ds5_15, ds0_5
        # Global_Functions.write_rasterio(arr, f'{local_path}/Siberia/PeatFire/BG_CC/data/SoilGrids/SoilGrids_{l}_v2.0.tif',
        #                                profile=profile)
        Global_Functions.write_rasterio(arr, f'/vsimem/{l}_{tile}_igh.tif',
                                       profile=profile)
        profile['transform'] = gt
        profile['height'] = 8000
        profile['width'] = 12000
        profile['crs'] = pyproj.CRS.from_epsg(4326)
        # print(profile)
        os.makedirs(f'{local_path}/Siberia/PeatFire/ByTile/{tile}/Predictors', exist_ok=True)
        Global_Functions.resample_by_ref(f'/vsimem/{l}_{tile}_igh.tif', 
                f'{local_path}/Siberia/PeatFire/ByTile/{tile}/Predictors/SoilGrids_{l}_{tile}_v2.0.tif',
                                profile=profile)


## 3) TerraClimate

In [ ]:
terraClimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
cems_ic = ee.ImageCollection('projects/climate-engine-pro/assets/ce-cems-fire-daily-4-1')
cems_avg_list = ee.data.listAssets('projects/ee-fireccihrba/assets/SFD_BA/Siberia/PeatFire/CEMS_ECMWF')['assets']
cems_avg = ee.ImageCollection(list(map(lambda x: ee.Image(x['id']), cems_avg_list)))
# cems_avg = ee.ImageCollection([ee.Image(x['id']) for x in cems_avg_list if 'DC' in x['id']])

def download_image(image, name, export_folder, geo, proj, scale, transform=None, to='Drive', logfile=False):
    if not logfile:
        logfile = io.StringIO()
    if transform:
        scale = None
    if to == 'Drive':
        task = ee.batch.Export.image.toDrive(image=image, 
                              description=name, 
                              folder=export_folder, 
                              crs=proj,  
                              scale=scale,
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)  
    else:
        task = ee.batch.Export.image.toAsset(image=image, 
                              description=name, 
                              assetId=f'{export_folder}/{name}', 
                              crs=proj,  
                              scale=scale, 
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)              
    task.start()
    Global_Functions.print_and_log(logfile, f"\t\t +++ Exporting {name} ...")


def get_lngTerm_stat(col, stat, months=[6, 9]):
    ## If you filter the year ypu will have a changing stats, not a worthed heavy processing
     # .filter(ee.Filter.calendarRange(int(year), int(year), 'year').Not()) 
    lngTerm = col.filter(ee.Filter.calendarRange(months[0], months[1], 'month')) \
        .map(lambda img: img.set('year', ee.Date(img.get('system:time_start')).get('year')))
    distYear = lngTerm.distinct('year')
    joinedCol = ee.Join.saveAll('sameYear').apply(distYear, lngTerm, 
                     ee.Filter.equals(**{'leftField': 'year', 'rightField': 'year'}))
    lngTerm_stat = ee.ImageCollection(joinedCol.map(lambda img: ee.ImageCollection.fromImages(img.get('sameYear')) \
                                .reduce(stat).set('year', img.get('year'))))
    return lngTerm_stat

def get_summer_stats(col, band, stat, date_start, date_end, lngTerm_stat=None):
    subset = col.select(band).filterDate(date_start, date_end)
    result = subset.reduce(stat)
    if not lngTerm_stat:
        lngTerm_stat = get_lngTerm_stat(col.select(band), stat, months=[6, 9])
    anomaly = result.subtract(lngTerm_stat.reduce(ee.Reducer.mean())).divide(lngTerm_stat.reduce(ee.Reducer.stdDev()))
    return result, anomaly

def get_cumulative_stats(col, band, stat, date_start, date_end):
    col = col.select(band)
    days_list = col.aggregate_array('system:index').getInfo()
    years_list = np.unique([i[:4] for i in days_list])
    years_list = ee.List(years_list.tolist())
    init_col = ee.ImageCollection([])
    def cumulative_stat(y, cum_col):
        # start = ee.Date(ee.Number.parse(y).format('%d-06-01'))
        # end = ee.Date(ee.Number.parse(y).format('%d-10-01'))
        start = ee.Date(date_start).update(year=ee.Number.parse(y))
        end = ee.Date(date_end).update(year=ee.Number.parse(y))
        image_stat = ee.ImageCollection(col).filterDate(start, end).reduce(stat).set('year', y)
        return ee.ImageCollection(cum_col).merge(image_stat)
    
    cumul_stat = ee.ImageCollection(years_list.iterate(cumulative_stat, init_col))
    result = cumul_stat.filter(ee.Filter.eq('year', date_start[:4])).first()
    anomaly = result.subtract(cumul_stat.reduce(ee.Reducer.mean())).divide(cumul_stat.reduce(ee.Reducer.stdDev()))   
    return result, anomaly

def get_annual(col, band, stat, date_start, date_end):
    lngTerm = col.filterDate(date_start, date_end).select(band)
    lngTermStat = lngTerm.reduce(stat).set('year', date_start[:4])
    return lngTermStat
    
    
def get_terraclimate(year, studyArea, tile=None, logfile=False):
    if not logfile:
        logfile = io.StringIO()

    if tile:    
        studyArea = tile.geometry()
        tilename = tile.get('TILE').getInfo()    
    date_start = f'{year}-06-01'
    date_end = f'{year}-10-01'
    
    """----- Main functions ------"""

    def get_yearly_mcwd(y, mcwd_lngterm):
        start = ee.Date(ee.Number(y).format('%d-01-01'))
        end = ee.Date(ee.Number(y).format('%d-10-01'))
        rain = terraClimate.select('pr').filterDate(start, end)
        evap = terraClimate.select('aet').filterDate(start, end).map(lambda image: image.multiply(0.1))
        acc_wd_rec = ee.ImageCollection([])
        
        def get_mcwd(aet, acc_wd):
            ide = aet.get('system:index')
            p = rain.filter(ee.Filter.eq('system:index', ide)).first()
            aet = evap.filter(ee.Filter.eq('system:index', ide)).first()
            water_deficit = aet.subtract(p).multiply(aet.gte(p))
            ## the same result as the following
            # water_deficit = aet.subtract(p).updateMask(aet.gte(p)).unmask(0)
            return ee.ImageCollection(acc_wd).merge(water_deficit)
        
        mcwd_rec = ee.ImageCollection(evap.iterate(get_mcwd, acc_wd_rec)).sum().rename('mcdw')
        mcwd_rec = mcwd_rec.set('system:time_start', start).set('system:time_end', end)
        return ee.ImageCollection(mcwd_lngterm).merge(mcwd_rec)
        

    """----- Processing ------"""

    bands = ['tmmx', 'tmmn', 'vpd', 'srad', 'pdsi', 'def', 'soil']
    layers = ['Tmax', 'Tmin', 'vpd', 'solar_down', 'pdsi', 'cwd', 'moisture']
    terra_stats = get_cumulative_stats(terraClimate, bands,
            ee.Reducer.mean(), date_start, date_end)
    
    
    # print(terra_stats[0].bandNames().getInfo())
    anomaly_layers = [f'{l}_anomaly' for l in layers]
    monthly_means = terra_stats[0].rename(layers)
    monthly_anomalies = terra_stats[1].rename(anomaly_layers)

    ## Drought Code
    bands_cems = ['duff_moisture_code', 'fine_fuel_moisture_code', 'fire_weather_index', 'drought_code']
    layers_cems = ['duff_moisture', 'finefuel_moisture', 'FWI', 'DC']
    cems_mean, cems_mean_anomaly = get_summer_stats(cems_ic, 
            bands_cems,  ee.Reducer.mean(), 
            date_start, date_end, lngTerm_stat=cems_avg)
    anomaly_layers_cems = [f'{l}_anomaly' for l in layers_cems]
    monthly_cems_means = cems_mean.rename(layers_cems)
    monthly_cems_anomalies = cems_mean_anomaly.rename(anomaly_layers_cems)    

    layers = {
              'TerraClimate_Means': monthly_means, 
              'TerraClimate_Anomalies': monthly_anomalies,
              'ECMWF_CEMS_Means': monthly_cems_means,
              'ECMWF_CEMS_Anomalies': monthly_cems_anomalies
             }
    for i in layers.items():
        if 'Terra' in i[0]:
            proj = terraClimate.first().projection().getInfo()
        else:
            proj = cems_ic.first().projection().getInfo()

        # print(proj)
        crs = 'EPSG:4326'
        gt = proj['transform']
        naming = f'{i[0]}_Siberia_{year}'
        if tile:
            naming = f'{i[0]}_Siberia_{year}_{tilename}'
        download_image(i[1], naming, 'PeatFire', studyArea, crs, scale=None, 
                   transform=gt, logfile=logfile)
    
  

In [ ]:
bands = ['tmmx', 'tmmn', 'vpd', 'srad', 'pdsi', 'def', 'soil']
layers = ['Tmax', 'Tmin', 'vpd', 'solar_down', 'pdsi', 'cwd', 'moisture']
Lngterm = get_lngTerm_stat(terraClimate.select(bands), ee.Reducer.mean(), months=[6, 9])
stats = {
        'LongTermMean': ee.Reducer.mean(),
        'LongTermSTD': ee.Reducer.stdDev()}
for l, s in stats.items(): 
    image = Lngterm.reduce(s).rename(layers)
    proj = terraClimate.first().projection().getInfo()
    crs = 'EPSG:4326'
    gt = proj['transform']
    naming = f'TerraClimate_{l}_Siberia'
    download_image(image, naming, 'PeatFire', studyArea, crs, scale=None, 
                   transform=gt, logfile=None) 

In [ ]:
bands = ['duff_moisture_code', 'fine_fuel_moisture_code', 'fire_weather_index', 'drought_code']
layers = ['DMC', 'FFMC', 'FWI', 'DC']
Lngterm = cems_avg
stats = {
        'LongTermMean': ee.Reducer.mean(),
        'LongTermSTD': ee.Reducer.stdDev()}
for l, s in stats.items(): 
    image = Lngterm.reduce(s).rename(layers).select(['DMC', 'FFMC', 'DC'])
    proj = cems_ic.first().projection().getInfo()
    crs = 'EPSG:4326'
    gt = proj['transform']
    naming = f'ECMWF_CEMS_{l}_Siberia'
    download_image(image, naming, 'PeatFire', studyArea, crs, scale=None, 
                   transform=gt, logfile=None) 

In [ ]:
Extent = ee.Geometry.BBox(63, 58, 180, 74)
Tiles = grid4d.aggregate_array('TILE').getInfo()
for year in list(range(2023, 1939, -1))[:]:
    print(year)
    get_terraclimate(year, Extent)

## 4) LST

In [ ]:
Tiles_MODIS_gee = ee.FeatureCollection("users/fireccihrba/BAMT/Regions/MODIS_GRID_WGS84")

def get_summer_stats(col, band, stat, date_start, date_end):
    subset = col.select(band).filterDate(date_start, date_end)
    result = subset.reduce(stat)
    lngTerm_stat = get_lngTerm_stat(col.select(band), stat, months=[7, 8])
    anomaly = result.subtract(lngTerm_stat.reduce(ee.Reducer.mean())).divide(lngTerm_stat.reduce(ee.Reducer.stdDev()))
    return result, anomaly

def get_cumulative_stats(col, band, stat, date_start, date_end):
    col = col.select(band)
    days_list = col.aggregate_array('system:index')#.getInfo()
    # years_list = np.unique([i[:4] for i in days_list])
    # years_list = ee.List(years_list.tolist())
    years_list = ee.List(days_list).map(lambda x: ee.String(x).slice(0, 4)).distinct()
    init_col = ee.ImageCollection([])
    def cumulative_stat(y, cum_col):
        start = ee.Date(ee.Number.parse(y).format('%d-07-01'))
        end = ee.Date(ee.Number.parse(y).format('%d-09-01'))
        single_year = ee.ImageCollection(col).filterDate(start, end).set('year', y).rename()
        stat_image = single_year.reduce(stat).rename('stat_image')#.set('system:index','2000_01_01') ## make fixed distinct date easy to pick
        single_year = single_year.merge(stat_image).toBands()
        return ee.ImageCollection(cum_col).merge(single_year)
    
    cumul_stat = ee.ImageCollection(years_list.iterate(cumulative_stat, init_col))
    overall_stat = cumul_stat.select('.*stat_image')
    LST_mean = overall_stat.filter(ee.Filter.eq('year', ee.String(date_start).slice(0, 4)))
    anomaly = LST_mean.subtract(overall_stat.reduce(ee.Reducer.mean())).divide(overall_stat.reduce(ee.Reducer.stdDev())) 
    print(LST_mean.getInfo())
    return LST_mean, anomaly

def download_image(image, name, export_folder, geo, proj, scale, transform=None, to='Drive', logfile=False):
    if transform:
        scale = None
    if to == 'Drive':
        task = ee.batch.Export.image.toDrive(image=image, 
                              description=name, 
                              folder=export_folder, 
                              crs=proj,  
                              scale=scale,
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)  
    else:
        task = ee.batch.Export.image.toAsset(image=image, 
                              description=name, 
                              assetId=f'{export_folder}/{name}', 
                              crs=proj,  
                              scale=scale, 
                              crsTransform=transform,
                              region=geo, 
                              maxPixels=1e13)              
    task.start()
    Global_Functions.print_and_log(logfile, f"\t\t +++ Exporting {name} ...")
    
def LST_anomaly(year, studyArea, tile=None, satellite='Terra', logfile=False):
    if not logfile:
        logfile = io.StringIO()

    if tile:    
        studyArea = tile.geometry()
        tilename = tile.get('TILE').getInfo()    
    date_start = f'{year}-07-01'
    date_end = f'{year}-09-01'  
    stat = ee.Reducer.mean()
    """----- Main functions ------"""    
    def preprocess_LST(image):
        rescaled = image.multiply(0.02)
        rescaled = rescaled.set('system:time_start', image.get('system:time_start')) 
        rescaled = rescaled.set('system:time_end', image.get('system:time_end'))
        return rescaled

    def filter_doy(d, deltamin, deltamax, y):
        doymax = int((datetime.strptime(f'{y+1}0101', '%Y%m%d') - timedelta(days=1)).strftime('%j'))
        doymax0 = int((datetime.strptime(f'{y}0101', '%Y%m%d') - timedelta(days=1)).strftime('%j'))
        if (d + deltamax) > doymax:
            filter1max = ee.Filter.dayOfYear(int(d - deltamin), doymax)
            filter2max = ee.Filter.dayOfYear(1, int(d + deltamax - doymax))
            result_filter = ee.Filter.Or(filter1max, filter2max)
        elif (d - deltamin) < 1:
            filter1min = ee.Filter.dayOfYear(int(d - deltamin + doymax0), doymax0)
            filter2min = ee.Filter.dayOfYear(1, int(d + deltamax))
            result_filter = ee.Filter.Or(filter1min, filter2min)
        else:
            result_filter = ee.Filter.dayOfYear(int(d - deltamin), int(d + deltamax))
        return result_filter
                                           
    """----- Processing ------"""
    
    if satellite == 'Aqua':
        ## 
        modis_11 = ee.ImageCollection("MODIS/061/MYD11A2")
    elif satellite == 'Terra':
        ## pass time 10:30 pm, most 
        modis_11 = ee.ImageCollection("MODIS/061/MOD11A2")
        
    modis_11 = modis_11.select('LST_Night_1km')
    days_list = modis_11.aggregate_array('system:index')
    ini = ee.List([])
    years_list = ee.List(days_list).map(lambda x: ee.String(x).slice(0, 4)).distinct()
    init_col = ee.ImageCollection([])
    def cumulative_stat(y, cum_col):
        start = ee.Date(ee.Number.parse(y).format('%d-06-01'))
        end = ee.Date(ee.Number.parse(y).format('%d-10-01'))
        single_year = ee.ImageCollection(modis_11).filterDate(start, end).toBands()
        doys = single_year.bandNames().map(lambda x: 
                   ee.Number.parse(ee.Date.parse('yyyy_MM_dd', ee.String(x).slice(0, 10)).format('D')))
        periods = doys.map(lambda x: ee.Number(x).divide(8).int())
        ini = ee.List([])
        names = periods.iterate(lambda x, ini: 
                   ee.List(ini).add(ee.String('image_').cat(ee.Number(x).format('%d'))), ini)
        
        single_year = single_year.rename(names)
        bandtype = single_year.select('image_19').bandTypes().get('image_19')
        single_year = ee.Image(ee.Algorithms.If(ee.String(y).compareTo('2001').eq(0), 
                  single_year.addBands(single_year.select(['image_20', 'image_22']).reduce('mean').rename('image_21').cast({'image_21': bandtype})), 
                                                single_year))
        stat_image = single_year.reduce(stat).rename('stat_image')
        single_year = single_year.addBands(stat_image).set('year', y)
        return ee.ImageCollection(cum_col).merge(single_year)
    
    cumul_stat = ee.ImageCollection(years_list.iterate(cumulative_stat, init_col))
    overall_stat = cumul_stat.select('.*stat_image')
    LST_mean = overall_stat.filter(ee.Filter.eq('year', ee.String(date_start).slice(0, 4))).first()
    anomaly = LST_mean.subtract(overall_stat.reduce(ee.Reducer.mean())).divide(overall_stat.reduce(ee.Reducer.stdDev())) 
    year_image = cumul_stat.filter(ee.Filter.eq('year', ee.String(date_start).slice(0, 4))).first()
    periods = year_image.select('[^.*stat].*').bandNames()
    init_col = ee.ImageCollection([])
    def get_period_anomaly(p, col):
        p = ee.String(p)
        subset = cumul_stat.select(p)
        subset_year = year_image.select(p)
        result = subset_year.subtract(subset.reduce(ee.Reducer.mean())).divide(subset.reduce(ee.Reducer.stdDev())) 
        result = result.where(result.gte(0.5), 1)
        result = result.where(result.lt(0.5), 0).rename('anomaly')
        return ee.ImageCollection(col).merge(result)
    
    period_anomaly = ee.ImageCollection(periods.iterate(get_period_anomaly, init_col)) 
    prop = period_anomaly.reduce(ee.Reducer.sum()).divide(period_anomaly.reduce(ee.Reducer.count()))
    
    ## Use the proper scale otherwise you might end up having slight misgeolocation
    LST_stat = LST_mean.rename('mean_LST')#.updateMask(BA_image.gt(0))
    LST_anomaly = anomaly.rename('LST_std_anomaly')#.updateMask(BA_image.gt(0))
    LST_persistence = prop.rename('LST_persistence')#.updateMask(BA_image.gt(0))
    layers = {
              'LST_Mean_JA': LST_stat, 
              'LST_std_anomaly_JA': LST_anomaly,
              'LST_persistence_JA': LST_persistence,
             }
    for i in layers.items():
        proj = modis_11.first().projection().getInfo()
        crs = proj['crs']
        crs = 'EPSG:4326'
        gt = proj['transform']
        scale = gt[0]
        print(crs, gt)
        naming = f'{i[0]}_Siberia_{year}'
        if tile:
            naming = f'{i[0]}_Siberia_{year}_{tilename}'
        download_image(i[1], naming, 'PeatFire', studyArea, crs, scale=scale, 
                   transform=None, logfile=logfile)



In [ ]:
Extent = ee.Geometry.BBox(63, 58, 180, 74)
Tiles = grid4d.aggregate_array('TILE').getInfo()
for year in list(range(2023, 2000, -1))[:]:
    print(year)
    LST_anomaly(year, Extent)

## 5) VCF 

In [ ]:
def get_VCF(year, studyArea, tile=None, logfile=False):
    if not logfile:
        logfile = io.StringIO()
    if tile:    
        studyArea = tile.geometry()
        tilename = tile.get('TILE').getInfo()    
    vcf = ee.ImageCollection("MODIS/006/MOD44B") \
          .filter(ee.Filter.date(f'{year}-01-01', f'{year+1}-01-01'))
    vcf = vcf.first().select(['Percent_Tree_Cover', 'Percent_NonTree_Vegetation']) \
                  .rename(['tree_cover', 'nontree_cover'])
    proj = vcf.projection().getInfo()
    crs = proj['crs']
    crs = 'EPSG:4326'
    gt = proj['transform']
    scale = gt[0]
    print(crs, gt)
    naming = f'MOD44B_VCF_Siberia_{year}'
    if tile:
        naming = f'MOD44B_VCF_Siberia_{year}_{tilename}'
    download_image(vcf, naming, 'PeatFire', studyArea, crs, scale=scale, 
               transform=None, logfile=logfile)    
    

In [ ]:
Extent = ee.Geometry.BBox(60, 58, 180, 74)
Tiles = grid4d.aggregate_array('TILE').getInfo()
for year in list(range(2016, 2000, -1))[:]:
    print(year)
    get_VCF(year, Extent)

## 6) ALOS DEM

In [ ]:
def get_DEM(studyArea, tile=None, logfile=False):
    if not logfile:
        logfile = io.StringIO()
    if tile:    
        studyArea = tile.geometry()
        tilename = tile.get('TILE').getInfo()    
    dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').filterBounds(studyArea)
    elevation = dataset.select('DSM')
    proj = elevation.first().select(0).projection()
    elevation_gee = elevation.mosaic().setDefaultProjection(proj).rename('elevation')
    bandtype = elevation_gee.bandTypes().get('elevation')
    # elevation_gee = elevation_gee.addBands(ee.Image.pixelLonLat())
    slope_gee = ee.Terrain.slope(elevation_gee).multiply(100).cast({'slope': bandtype})
    aspect_gee = ee.Terrain.aspect(elevation_gee).convolve(ee.Kernel.square(1, 'pixels')).cast({'aspect': bandtype})
    topo = elevation_gee.addBands(slope_gee).addBands(aspect_gee)
    BA_image = BA30.qualityMosaic('b1')
    # print(BA_image.getInfo())
    topo = topo.updateMask(BA_image.gt(0))
    print(topo.getInfo())
    proj = proj.getInfo()
    crs = proj['crs']
    gt = proj['transform']
    print(crs, gt)
    naming = f'ALOSDEM30_Siberia'
    if tile:
        naming = f'ALOSDEM30_Siberia_{tilename}'
    download_image(topo, naming, 'PeatFire', studyArea, crs, scale=None, 
               transform=gt, logfile=logfile)    

In [ ]:
Extent = ee.Geometry.BBox(60, 58, 180, 74)
Tiles = grid4d.aggregate_array('TILE').getInfo()
# tile = 'TILE-66N108E'
for tile in Tiles[:]:
    tile = grid4d.filter(ee.Filter.eq('TILE', tile)).first()
    get_DEM(Extent, tile)

## 7)  AGB

In [ ]:
def get_agb(studyArea, tile=None, logfile=False):
    if not logfile:
        logfile = io.StringIO()
    if tile:    
        studyArea = tile.geometry()
        tilename = tile.get('TILE').getInfo()  
        
    AGB = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB").first().select('AGB')    ##2010
    BA_image = BA30.qualityMosaic('b1')
    AGB = AGB.updateMask(BA_image.gt(0).focalMax(kernel=ee.Kernel.euclidean(4, units='pixels'), iterations=1))
    proj = AGB.projection().getInfo()
    crs = proj['crs']
    gt = proj['transform']
    print(crs, gt)
    naming = f'AGB2010_Siberia'
    if tile:
        naming = f'{naming}_{tilename}'
    download_image(AGB, naming, 'PeatFire', studyArea, crs, scale=None, 
               transform=gt, logfile=logfile)   
    

In [ ]:
Extent = ee.Geometry.BBox(63, 58, 180, 74)
get_agb(Extent, tile=None)

## Resampling

In [ ]:
def resample_vars(layer, tile, Tiles4d, input_path, local_path, tiled, 
                  yearly, year, logfile=False):
    if not logfile:
        logfile = io.StringIO()
    Global_Functions.print_and_log(logfile, f'\n\t\t{15 * "-"} {tile} {15 * "-"}')   

    ref = f'{local_path}/ByTile/{tile}/Enhanced/Yearly/BAMT_BA_Siberia_Lndst_{year}_{tile}_JD_Correct_Patches.tif'
    with rio.open(ref) as src:
        profile = src.profile
    lon = int(tile[8:11])
    lat = int(tile[5:7])
    
    if tiled:
        lon6d = lon if lon in np.array(Tiles4d)[:, 1] else lon-3
        lat4d = lat if lat in np.array(Tiles4d)[:, 0] else lat+2
        filenames = glob.glob(f"{input_path}/{layer}/{layer}*" + yearly*f"_{year}" + f"_TILE-{lat4d:02}N{lon6d:03}E*.tif")
    else:
        filenames = glob.glob(f"{input_path}/{layer}/{layer}*" + yearly*f"_{year}*" + f".tif")

    with rio.open(filenames[0]) as src:
        profile['dtype'] = src.profile['dtype']
    naming = f"{layer}_Siberia" + yearly*f"_{year}" + f"_{tile}"
    Global_Functions.mosaic_gdal(naming, files=filenames, pattern='', inputPath='', 
            outPath=f'{local_path}/PeatFire/ByTile/{tile}/Predictors', vrtPath='/vsimem', profile=profile)  
    Global_Functions.print_and_log(logfile, f'\t\t+++++ {naming} is saved +++++\n')
        
    # return {'tile': tile, 'text': logfile.getvalue()}

In [ ]:
local_path = '/media/amin/STORAGE/STORAGE/OneDrive/PhD/Landsat_BA'
raw_path = '/media/amin/STORAGE/STORAGE/OneDrive/PhD/Landsat_BA/Siberia/PeatFire/BG_CC/data'
res_path = '/media/amin/DISK6T/PhD/Landsat_BA/Siberia'
sib4d = gpd.read_file(f'{local_path}/Regions/Tiles_Siberia_4dLat.shp')
sib3d = gpd.read_file(f'{local_path}/Regions/Tiles_Siberia_3dLon.shp')
Tiles4d = [(int(i[5:7]), int(i[8:11]))  for i in sib4d.TILE]
Tiles = ['TILE-' + i for i in sib3d.TILE]

# LogFile = open(f"{res_path}/PeatFire/Logs/Logfile_Siberia_PeatFire_Predictors_Resampling.txt", 'w')
LogFile = io.StringIO()
Global_Functions.print_and_log(LogFile, 
            f'{30 * "-"} Logfile of PeatFire Predictors Resampling {30 * "-"}\n\n')
layer_args = dict(
            MOD44B_VCF = {'tiled': False, 'yearly': True, 'years': range(2001, 2021)},
            LST_Mean = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            LST_std_anomaly = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            LST_persistence = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)}, 
            TerraClimate_Means = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            TerraClimate_Anomalies = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            ECMWF_CEMS_Means = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            ECMWF_CEMS_Anomalies = {'tiled': False, 'yearly': True, 'years': range(2001, 2024)},
            ALOSDEM30 = {'tiled': True, 'yearly': False, 'years': [2021]},
            NBR = {'tiled': True, 'yearly': True, 'years': range(2001, 2024)},
            AGB2010 = {'tiled': False, 'yearly': False, 'years': [2021]}
            )
time0 = datetime.now()
for year in range(2023, 2000, -1):
    Global_Functions.print_and_log(LogFile, 
           f'\n{10 * "#"} Processing of year {year} {10 * "#"}\n\n')
    time1 = datetime.now()
    for layer, k in layer_args.items():
        if year in k['years']:
            kwargs = k.copy()
            kwargs.pop('years', None)
            Global_Functions.print_and_log(LogFile, f'\n\t\t{5 * "*"} {layer}')   
            listdict = Parallel(n_jobs=12, verbose=100, backend='threading') (delayed(resample_vars) (
                    layer, tile, Tiles4d, raw_path, res_path, **kwargs) for tile in Tiles[:]) 
            listdict.sort(key=operator.itemgetter('tile'))
            [LogFile.write(listdict[i]['text']) for i in range(len(listdict))]
            for i, tile in enumerate(sorted(Tiles)):
                print(i)
                resample_vars(layer, tile, Tiles4d, raw_path, res_path, year=year, logfile=LogFile, **kwargs)
    time2 = datetime.now()
    delta = time2 - time1
    hours, minutes, seconds = Global_Functions.convert_timedelta(delta)
    Global_Functions.print_and_log(LogFile, '\n%s Year %s is processed in %s hours, %s minutes, %s seconds %s '
                                  %(10 *'#', year, hours, minutes, seconds, 10 * '#'))    
timef = datetime.now()
delta = timef - time0    
hours, minutes, seconds = Global_Functions.convert_timedelta(delta)
Global_Functions.print_and_log(LogFile, '\n\n%s FINISHED in %s hours, %s minutes, %s seconds %s'
                          %(20 *'-', hours, minutes, seconds, 20 * '-'))    
LogFile.close()